In [2]:
from src.config import Settings
from src.services.parser.factory import make_parser_service

In [3]:
parser_client = make_parser_service(settings=Settings())

In [5]:
pdf_file = 'assets/testpaper.pdf'
result = await parser_client.parse_document(pdf_file)

2025-11-05 23:53:12,796 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-05 23:53:12,839 - INFO - Going to convert document batch...
2025-11-05 23:53:12,840 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 82671889753a65953bfba9f907b47bed
2025-11-05 23:53:12,846 - INFO - Loading plugin 'docling_defaults'
2025-11-05 23:53:12,848 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-05 23:53:12,854 - INFO - Loading plugin 'docling_defaults'
2025-11-05 23:53:12,858 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-05 23:53:12,882 - INFO - Accelerator device: 'mps'
2025-11-05 23:53:14,553 - INFO - Accelerator device: 'mps'
2025-11-05 23:53:15,118 - INFO - Processing document testpaper.pdf


len(pages)=1, 0-0
len(valid_pages)=1
len(valid_page_images)=1
len(pages)=4, 1-4
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 5-8
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 9-12
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 13-16
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 17-20
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=4, 21-24
len(valid_pages)=4
len(valid_page_images)=4
len(pages)=1, 25-25
len(valid_pages)=1
len(valid_page_images)=1


2025-11-05 23:53:25,527 - INFO - Finished converting document testpaper.pdf in 12.73 sec.
/var/folders/2k/0py09_714jx8schwf06rqfpm0000gn/T/ipykernel_10819/360203371.py:2: RuntimeWarning: coroutine 'ParserService.parse_document' was never awaited
  result = await parser_client.parse_document(pdf_file)


In [8]:
parsed_doc = result.doc_content

In [20]:
for section in parsed_doc.sections:
    print(section.title)

Content
AI Open
Graph neural networks: A review of methods and applications
1. Introduction
https://doi.org/10.1016/j.aiopen.2021.01.001
2. General design pipeline of GNNs
2.1. Find graph structure
2.2. Specify graph type and scale
2.3. Design loss function
2.4. Build model using computational modules
3. Instantiations of computational modules
3.1. Propagation modules - convolution operator
3.1.1. Spectral approaches
3.1.2. Basic spatial approaches
3.1.3. Attention-based spatial approaches
3.1.4. General frameworks for spatial approaches
3.2. Propagation modules - recurrent operator
3.2.1. Convergence-based methods
3.2.2. Gate-based methods
3.3. Propagation modules - skip connection
3.4. Sampling modules
3.4.1. Node sampling
3.4.2. Layer sampling
3.4.3. Subgraph sampling
3.5. Pooling modules
3.5.1. Direct pooling modules
3.5.2. Hierarchical pooling modules
4. Variants considering graph type and scale
4.1. Directed graphs
4.2. Heterogeneous graphs
4.2.1. Meta-path-based methods
4.2.2. E